In [1]:
# look at tools/set_up_magics.ipynb
yandex_metrica_allowed = True ; get_ipython().run_cell('# one_liner_str\n\nget_ipython().run_cell_magic(\'javascript\', \'\', \'// setup cpp code highlighting\\nIPython.CodeCell.options_default.highlight_modes["text/x-c++src"] = {\\\'reg\\\':[/^%%cpp/]} ;\')\n\n# creating magics\nfrom IPython.core.magic import register_cell_magic, register_line_magic\nfrom IPython.display import display, Markdown, HTML\nimport argparse\nfrom subprocess import Popen, PIPE\nimport random\nimport sys\nimport os\nimport re\nimport signal\nimport shutil\nimport shlex\nimport glob\n\n@register_cell_magic\ndef save_file(args_str, cell, line_comment_start="#"):\n    parser = argparse.ArgumentParser()\n    parser.add_argument("fname")\n    parser.add_argument("--ejudge-style", action="store_true")\n    args = parser.parse_args(args_str.split())\n    \n    cell = cell if cell[-1] == \'\\n\' or args.no_eof_newline else cell + "\\n"\n    cmds = []\n    with open(args.fname, "w") as f:\n        f.write(line_comment_start + " %%cpp " + args_str + "\\n")\n        for line in cell.split("\\n"):\n            line_to_write = (line if not args.ejudge_style else line.rstrip()) + "\\n"\n            if line.startswith("%"):\n                run_prefix = "%run "\n                if line.startswith(run_prefix):\n                    cmds.append(line[len(run_prefix):].strip())\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                run_prefix = "%# "\n                if line.startswith(run_prefix):\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                raise Exception("Unknown %%save_file subcommand: \'%s\'" % line)\n            else:\n                f.write(line_to_write)\n        f.write("" if not args.ejudge_style else line_comment_start + r" line without \\n")\n    for cmd in cmds:\n        display(Markdown("Run: `%s`" % cmd))\n        get_ipython().system(cmd)\n\n@register_cell_magic\ndef cpp(fname, cell):\n    save_file(fname, cell, "//")\n\n@register_cell_magic\ndef asm(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef makefile(fname, cell):\n    assert not fname\n    save_file("makefile", cell.replace(" " * 4, "\\t"))\n        \n@register_line_magic\ndef p(line):\n    try:\n        expr, comment = line.split(" #")\n        display(Markdown("`{} = {}`  # {}".format(expr.strip(), eval(expr), comment.strip())))\n    except:\n        display(Markdown("{} = {}".format(line, eval(line))))\n        \ndef show_file(file, clear_at_begin=True, return_html_string=False):\n    if clear_at_begin:\n        get_ipython().system("truncate --size 0 " + file)\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                elem.innerText = xmlhttp.responseText;\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (errors___OBJ__ < 10 && !entrance___OBJ__) {\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n        \n        <font color="white"> <tt>\n        <p id="__OBJ__" style="font-size: 16px; border:3px #333333 solid; background: #333333; border-radius: 10px; padding: 10px;  "></p>\n        </tt> </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__ -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n    \nBASH_POPEN_TMP_DIR = "./bash_popen_tmp"\n    \ndef bash_popen_terminate_all():\n    for p in globals().get("bash_popen_list", []):\n        print("Terminate pid=" + str(p.pid), file=sys.stderr)\n        p.terminate()\n    globals()["bash_popen_list"] = []\n    if os.path.exists(BASH_POPEN_TMP_DIR):\n        shutil.rmtree(BASH_POPEN_TMP_DIR)\n\nbash_popen_terminate_all()  \n\ndef bash_popen(cmd):\n    if not os.path.exists(BASH_POPEN_TMP_DIR):\n        os.mkdir(BASH_POPEN_TMP_DIR)\n    h = os.path.join(BASH_POPEN_TMP_DIR, str(random.randint(0, 1e18)))\n    stdout_file = h + ".out.html"\n    stderr_file = h + ".err.html"\n    run_log_file = h + ".fin.html"\n    \n    stdout = open(stdout_file, "wb")\n    stdout = open(stderr_file, "wb")\n    \n    html = """\n    <table width="100%">\n    <colgroup>\n       <col span="1" style="width: 70px;">\n       <col span="1">\n    </colgroup>    \n    <tbody>\n      <tr> <td><b>STDOUT</b></td> <td> {stdout} </td> </tr>\n      <tr> <td><b>STDERR</b></td> <td> {stderr} </td> </tr>\n      <tr> <td><b>RUN LOG</b></td> <td> {run_log} </td> </tr>\n    </tbody>\n    </table>\n    """.format(\n        stdout=show_file(stdout_file, return_html_string=True),\n        stderr=show_file(stderr_file, return_html_string=True),\n        run_log=show_file(run_log_file, return_html_string=True),\n    )\n    \n    cmd = """\n        bash -c {cmd} &\n        pid=$!\n        echo "Process started! pid=${{pid}}" > {run_log_file}\n        wait ${{pid}}\n        echo "Process finished! exit_code=$?" >> {run_log_file}\n    """.format(cmd=shlex.quote(cmd), run_log_file=run_log_file)\n    # print(cmd)\n    display(HTML(html))\n    \n    p = Popen(["bash", "-c", cmd], stdin=PIPE, stdout=stdout, stderr=stdout)\n    \n    bash_popen_list.append(p)\n    return p\n\n\n@register_line_magic\ndef bash_async(line):\n    bash_popen(line)\n    \n    \ndef show_log_file(file, return_html_string=False):\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function halt__OBJ__(elem, color)\n        {\n            elem.setAttribute("style", "font-size: 14px; background: " + color + "; padding: 10px; border: 3px; border-radius: 5px; color: white; ");                    \n        }\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            if (entrance___OBJ__ < 0) {\n                entrance___OBJ__ = 0;\n            }\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                if (elem.innerHTML != xmlhttp.responseText) {\n                                    elem.innerHTML = xmlhttp.responseText;\n                                }\n                                if (elem.innerHTML.includes("Process finished.")) {\n                                    halt__OBJ__(elem, "#333333");\n                                } else {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                }\n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (!entrance___OBJ__) {\n                                if (errors___OBJ__ < 6) {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                } else {\n                                    halt__OBJ__(elem, "#994444");\n                                }\n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n\n        <p id="__OBJ__" style="font-size: 14px; background: #000000; padding: 10px; border: 3px; border-radius: 5px; color: white; ">\n        </p>\n        \n        </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__.md -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n\n    \nclass TInteractiveLauncher:\n    tmp_path = "./interactive_launcher_tmp"\n    def __init__(self, cmd):\n        try:\n            os.mkdir(TInteractiveLauncher.tmp_path)\n        except:\n            pass\n        name = str(random.randint(0, 1e18))\n        self.inq_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".inq")\n        self.log_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".log")\n        \n        os.mkfifo(self.inq_path)\n        open(self.log_path, \'w\').close()\n        open(self.log_path + ".md", \'w\').close()\n\n        self.pid = os.fork()\n        if self.pid == -1:\n            print("Error")\n        if self.pid == 0:\n            exe_cands = glob.glob("../tools/launcher.py") + glob.glob("../../tools/launcher.py")\n            assert(len(exe_cands) == 1)\n            assert(os.execvp("python3", ["python3", exe_cands[0], "-l", self.log_path, "-i", self.inq_path, "-c", cmd]) == 0)\n        self.inq_f = open(self.inq_path, "w")\n        interactive_launcher_opened_set.add(self.pid)\n        show_log_file(self.log_path)\n\n    def write(self, s):\n        s = s.encode()\n        assert len(s) == os.write(self.inq_f.fileno(), s)\n        \n    def get_pid(self):\n        n = 100\n        for i in range(n):\n            try:\n                return int(re.findall(r"PID = (\\d+)", open(self.log_path).readline())[0])\n            except:\n                if i + 1 == n:\n                    raise\n                time.sleep(0.1)\n        \n    def input_queue_path(self):\n        return self.inq_path\n        \n    def close(self):\n        self.inq_f.close()\n        os.waitpid(self.pid, 0)\n        os.remove(self.inq_path)\n        # os.remove(self.log_path)\n        self.inq_path = None\n        self.log_path = None \n        interactive_launcher_opened_set.remove(self.pid)\n        self.pid = None\n        \n    @staticmethod\n    def terminate_all():\n        if "interactive_launcher_opened_set" not in globals():\n            globals()["interactive_launcher_opened_set"] = set()\n        global interactive_launcher_opened_set\n        for pid in interactive_launcher_opened_set:\n            print("Terminate pid=" + str(pid), file=sys.stderr)\n            os.kill(pid, signal.SIGKILL)\n            os.waitpid(pid, 0)\n        interactive_launcher_opened_set = set()\n        if os.path.exists(TInteractiveLauncher.tmp_path):\n            shutil.rmtree(TInteractiveLauncher.tmp_path)\n    \nTInteractiveLauncher.terminate_all()\n   \nyandex_metrica_allowed = bool(globals().get("yandex_metrica_allowed", False))\nif yandex_metrica_allowed:\n    display(HTML(\'\'\'<!-- YANDEX_METRICA_BEGIN -->\n    <script type="text/javascript" >\n       (function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)};\n       m[i].l=1*new Date();k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)})\n       (window, document, "script", "https://mc.yandex.ru/metrika/tag.js", "ym");\n\n       ym(59260609, "init", {\n            clickmap:true,\n            trackLinks:true,\n            accurateTrackBounce:true\n       });\n    </script>\n    <noscript><div><img src="https://mc.yandex.ru/watch/59260609" style="position:absolute; left:-9999px;" alt="" /></div></noscript>\n    <!-- YANDEX_METRICA_END -->\'\'\'))\n\ndef make_oneliner():\n    html_text = \'("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "")\'\n    html_text += \' + "<""!-- MAGICS_SETUP_PRINTING_END -->"\'\n    return \'\'.join([\n        \'# look at tools/set_up_magics.ipynb\\n\',\n        \'yandex_metrica_allowed = True ; get_ipython().run_cell(%s);\' % repr(one_liner_str),\n        \'display(HTML(%s))\' % html_text,\n        \' #\'\'MAGICS_SETUP_END\'\n    ])\n       \n\n');display(HTML(("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "") + "<""!-- MAGICS_SETUP_PRINTING_END -->")) #MAGICS_SETUP_END

<IPython.core.display.Javascript object>

# Вступление: Linux, командная строка, Jupyter notebook

Возможно кому-то Jupyter notebook покажется лишним в этом ряду, но так случилось, что я буду вести у вас АКОС, а мне он кажется очень удобным инструментом :) И вы в будущем все равно с ним столкнетесь на других курсах.

<table width=100%> <tr>
    <th width=20%> <b>Видеозапись семинара &rarr; </b> </th>
    <th>
    <a href="https://www.youtube.com/watch?v=E0lg8pzzR7o&list=PLjzMm8llUm4AmU6i_hPU0NobgA4VsBowc">
        <img src="video.png" width="320"  height="160" align="left" alt="Видео с семинара"> 
    </a>
    </th>
    <th> </th>
</tr> </table>


[Ридинг Яковлева](https://github.com/victor-yacovlev/mipt-diht-caos/blob/master/practice/linux_basics/intro.md) **И сейчас, и в будущем: читайте эти ридинги, там много полезной информации. Я стараюсь ее не дублировать, использование моих ноутбуков подразумевает чтение ридингов Яковлева.**


Сегодня в программе:
* <a href="https://github.com/victor-yacovlev/mipt-diht-caos/blob/master/practice/linux_basics/intro.md" style="color:#856024"> Очень кратко о Linux </a>
* <a href="#terminal" style="color:#856024"> Часто используемые команды терминала </a>
  * <a href="#task1" style="color:#856024"> Задача 1 </a>
  * <a href="#task2" style="color:#856024"> Задача 2 </a>
  * <a href="#task3" style="color:#856024"> Задача 3 </a>
* <a href="#jupyter" style="color:#856024"> Особенности Jupyter notebok используемые в курсе </a>


## <a name="linux"></a> Очень кратко о Linux

Читайте [ридинг Яковлева](https://github.com/victor-yacovlev/mipt-diht-caos/blob/master/practice/linux_basics/intro.md). Там неплохо написано про линукс и не только.

Еще есть [конспект семинара Александра Пономарева](https://github.com/Alexponomarev7/caos_seminars/tree/master/sem1).

И куча статей в интернете :)

Если вы все это знаете, можете поиграть в [bandit](https://overthewire.org/wargames/bandit/). Сложненько, но интересно

## <a name="terminal"></a> Часто используемые команды терминала

Первая важная особенность Jupyter notebook: можно запускать консольные команды, добавляя `!` в начало.
То есть `!echo Hello` запустит `echo Hello` примерно так же, как это происходило бы в консоли.

### `cd`, `pwd`, `cp`, `mv`

`cd` - команда оболочки `bash`

`export PWD=/home` - альтернатива `cd`

Почти все остальные часто используемые команды - на самом деле запускаемые программы.


### `echo`

In [2]:
# Вывод строки
!echo Hello

Hello


### `>`, `>>`, `cat`

In [123]:
# Перенаправление вывода в файл
!echo "Hello1" > file.txt
!echo "Hello2" > file.txt # Файл перезапишется
# Перенаправление вывода в файл (при этом файл дозапишется, а не перезапишется)
!echo "Hello3" >> file.txt

# Вывод файла 
!cat file.txt

Hello2
Hello3


In [70]:
! >file.txt echo "Hello1" # И так тоже можно :)
!cat file.txt

Hello1


Еще особенность Jupyter notebook: можно делать ячейки "магическими", добавляя `%%bash`, `%%python`, `%%time` в начало.

Магия `%%bash` запустит ячейку как bash-скрипт. А bash-скрипт это почти то же самое, что последовательность команд в консоли.

In [125]:
%%bash 
exec 2>&1 # Настройка bash, чтобы все что пишется в поток ошибок на самом деле писалось в стандартный поток
set -x # Настройка bash, чтобы он выводил все исполняемые команды в поток ошибок

# -e включает восприятие \n как перевода строки у команды echo
echo -e "Hello1\nHello2" > file.txt
cat file.txt

+ echo -e 'Hello1\nHello2'
+ cat file.txt
Hello1
Hello2


### `|`, `<`

In [68]:
%%bash 
exec 2>&1 ; set -x # Команды можно писать в одну строчку разделяя с помощью ;

echo -e "Hello1\nHello2\nHello10" > file.txt
# Можно направлять вывод одной команды на вход другой с помощью |
# Команда grep в таком использовании читает все строки из входа, 
#   и выводит только содержащие подстроку`o1`
cat file.txt | grep o1

<file.txt grep llo2
grep llo2 < file.txt

+ echo -e 'Hello1\nHello2\nHello10'
+ cat file.txt
+ grep o1
Hello1
Hello10
+ grep llo2
Hello2
+ grep llo2
Hello2


In [111]:
%%bash 
# Можно объединять команды с помощью &&
# Тогда вторая выполнится только, если успешно выполнилась первая (как и в C/C++)
echo -n "Hello " && echo world!
echo -----------------
echo -n "Hello " || echo world! ; echo
echo -----------------
echo -n "Hello " && echBUG1o -n jail && echo -n freedom! ; echo
echo -----------------
echo -n "Hello " && echBUG1o jail || echo freedom!

Hello world!
-----------------
Hello 
-----------------
Hello 
-----------------
Hello freedom!


bash: line 7: echBUG1o: command not found
bash: line 9: echBUG1o: command not found


In [16]:
%%bash 
exec 2>&1 ; set -x
# Создадим пустой файл
touch a.txt
# Выведем список файлов в папке
ls *.txt
# Удалим файл
rm a.txt && echo "rm success" || echo "rm fail"
rm a.txt && echo "rm success" || echo "rm fail"

+ touch a.txt
+ ls a.txt b.txt file2.txt file.txt
a.txt
b.txt
file2.txt
file.txt
+ rm a.txt
+ echo 'rm success'
rm success
+ rm a.txt
rm: cannot remove 'a.txt': No such file or directory
+ echo 'rm fail'
rm fail


In [107]:
%%bash 
exec 2>&1 ; set -x
# Начало, конец файла, количество строк
echo -e "1\n2\n3\n4" > a.txt
cat a.txt | head -n 2
cat a.txt | tail -n 2
cat a.txt | wc -l

+ echo -e '1\n2\n3\n4'
+ cat a.txt
+ head -n 2
1
2
+ cat a.txt
+ tail -n 2
3
4
+ cat a.txt
+ wc -l
4


In [ ]:
# Получить документацию по программе легко с помощью команды man
!man head | head -n 10

А, важный момент, вообще по умолчанию в ячейках Jupyter notebook пишется код на том языке, с ядром которого запущен ноутбук. Сейчас это python3

In [63]:
with open("b.txt", "w") as f:
    for i in range(100):
        tup = (str(i), "div17" if i % 17 == 0 else "no17")
        f.write("\t".join(tup) + "\n")

## <a name="task1"></a> Задача 1

In [69]:
%%bash
cat b.txt | head -n 5
# Выведите все строчки где есть подстрока div17
# <your line of code here>

0	div17
1	no17
2	no17
3	no17
4	no17


## <a name="task2"></a> Задача 2

In [81]:
%%bash
# Выясните, что выведет следующая команда, не запуская ее.
# echo -e "A\nB\nC\nD" | grep -v C -n 
# Используйте только команды man и grep

In [99]:
%%bash
# Подсказка
man grep | grep -e "-C" -C 2

              option, this has no effect and a warning is given.

       -C NUM, -NUM, --context=NUM
              Print NUM lines of output context.  Places a line  containing  a
              group separator (--) between contiguous groups of matches.  With


## <a name="task3"></a> Задача 3

In [131]:
%%bash

# Создадим файл на 100 строк
( for i in {0..99} ; do echo line$i ; done ) > a.txt
cat a.txt | head -n 3

# А теперь напишите последовательность команд, которая заменит вторую строчку с line1 на line100
# нельзя использовать replace конструкции, вам достаточно команд, которые были перечислены ранее :)

# <your code here>

line0
line1
line2


<details> <summary> Способ в лоб (пожалуйста не смотрите, пока не напишете код решения задачки) </summary>
  <pre> <code> cat a.txt | python2 -c 'import sys; lines = list(sys.stdin); lines[1] = "line100\n"; print "".join(lines)' </code> </pre>
</details>

### Стандартные потоки ввода (stdin, 0), вывода (stdout, 1), ошибок (stderr, 2)

По умолчанию, когда мы запускаем программу, то у нее есть три стандартных потока: 
* ввода (через него все что мы печатаем в терминале передается программе) 
* вывода (через него то, что выводит программа (printf, std::cout) попадает в терминал)
* ошибок (примерно то же, что и stdout, но по-другому исползуется)

In [81]:
%%bash
echo Hello # Успешное завершение в вывод текста в stdout
rm not_existent_file # Ошибка, и текст пишется в поток stderr
true # Просто успешно завершающаяся команда, чтобы скрипт не завершился ошибкой.

Hello


rm: cannot remove 'not_existent_file': No such file or directory


Потоки ввода можно перенаправлять в файлы:

In [97]:
%%bash
echo Hello 1> out.txt 2> err.txt 
echo "stdout: \"`cat out.txt`\" stderr: \"`cat err.txt`\"" 
rm not_existent_file 1> out.txt 2> err.txt 
echo "stdout: \"`cat out.txt`\" stderr: \"`cat err.txt`\""

echo "Hello stXdents!" > file.txt
python2 -c "import sys; print list(sys.stdin)[0].replace('stXdents', 'students')" 0< file.txt
python2 -c "import os; print os.read(10, 100)" 10< file.txt # подумайте, что бы это могло быть :)

stdout: "Hello" stderr: ""
stdout: "" stderr: "rm: cannot remove 'not_existent_file': No such file or directory"
Hello students!

Hello stXdents!



При этом `>` синоним к `>1` (аналогично `>>`). А `0<` то же самое что и `<`.

### `ps`, `top`, `kill`, `killall`, `pidof` - найти, убить, убить попроще, найти по имени

In [112]:
%%bash
ps aux | grep ipyk

pechatn+   83544  0.0  2.4 552820 50372 ?        Ssl  06:42   0:06 /usr/bin/python3 -m ipykernel_launcher -f /home/pechatnov/.local/share/jupyter/runtime/kernel-b516c8c2-9d61-4427-b1e7-4093e713084b.json
pechatn+   83634  0.0  2.3 550660 47524 ?        Ssl  06:49   0:02 /usr/bin/python3 -m ipykernel_launcher -f /home/pechatnov/.local/share/jupyter/runtime/kernel-ef4f098b-5748-4e02-b2dc-8b169c68baca.json
pechatn+   89374  0.1  2.3 551588 48756 ?        Ssl  14:07   0:03 /usr/bin/python3 -m ipykernel_launcher -f /home/pechatnov/.local/share/jupyter/runtime/kernel-b9d9533d-f121-4a84-af61-4a855dd1c410.json
pechatn+   89824  0.0  0.0  17668   732 ?        S    14:58   0:00 grep ipyk


In [140]:
%%cpp bad_program.cpp
%run g++ -O3 -Os bad_program.cpp -o bad_program.exe
int main() { while (1) {} }

Run: `g++ -O3 -Os bad_program.cpp -o bad_program.exe`

`TInteractiveLauncher` - моя магическая штука для запуска программ в интерактивном режиме из Jupyter notebook

In [141]:
a = TInteractiveLauncher("./bad_program.exe")

In [126]:
get_ipython().system("ps aux | grep bad_prog")

pechatn+   89956  0.3  0.5  28624 11568 ?        S    15:10   0:01 python3 ../tools/launcher.py -l ./interactive_launcher_tmp/38653968711948138.log -i ./interactive_launcher_tmp/38653968711948138.inq -c ./bad_program.exe
pechatn+   89957 99.8  0.0   2356   516 ?        R    15:10   7:43 ./bad_program.exe
pechatn+   90003  0.0  0.0   2608   600 pts/2    Ss+  15:17   0:00 /usr/bin/sh -c ps aux | grep bad_prog
pechatn+   90005  0.0  0.0  17668   664 pts/2    S+   15:17   0:00 grep bad_prog


In [139]:
get_ipython().system("kill -9 " + str(a.get_pid()))
a.close()

In [58]:
a = TInteractiveLauncher("./bad_program.exe")
get_ipython().system("pidof bad_program.exe")
get_ipython().system("killall -9 bad_program.exe")
a.close()

83766


## <a name="jupyter"></a> Особенности Jupyter notebok используемые в курсе

In [130]:
# код на питоне
[i for i in range(0, 3000, 17) if str(i).endswith("19")] # просто что-то странное

[119, 1819]

In [132]:
%%bash
echo "Bash code"

Bash code


Jupyter notebook позволяет определять собственную магию. И в первой ячейке каждого моего ноутбука есть однострочник, который определяет несколько магий:

In [136]:
# Просто вывод строки и ее значения как выражения в питоне
a = 1; b = 2
%p a + b # Sum of a and b
%p (a, b)

`a + b = 3`  # Sum of a and b

(a, b) = (1, 2)

In [140]:
%%save_file file.txt 
%# Сохраняет ячейку как файл, комментируя загаловок (А в этой строке просто комментарий)
%run cat file.txt # Выполняет команды следующие после %run в заголовках ячейки
Содержимое файла

Run: `cat file.txt # Выполняет команды следующие после %run в заголовках ячейки`

# %%cpp file.txt 
# %# Сохраняет ячейку как файл
# %run cat file.txt # Выполняет команды следующие после %run в заголовках ячейки
Содержимое файла



In [141]:
%%save_file a.sh
%run bash a.sh

echo 123

Run: `bash a.sh`

123


In [152]:
%%cpp a.cpp
%# По большому счету тот же save_file, но подсвечивает синтаксис C++
%run g++ a.cpp -o a.exe
%run ./a.exe

#include <iostream>

int main() {
    std::cout << "Hello world!" << std::endl;
}

Run: `g++ a.cpp -o a.exe`

Run: `./a.exe`

Hello world!


In [144]:
!cat a.cpp

// %%cpp a.cpp
// %# По большому счету тот же save_file, но подсвечивает синтаксис C++
// %run g++ a.cpp -o a.exe
// %run ./a.exe

#include <iostream>

int main() {
    std::cout << "Hello world!" << std::endl;
}



In [107]:
%%cpp b.cpp
%run g++ b.cpp -o b.exe

#include <iostream>

int main() {
    std::string s;
    std::cin >> s;
    std::cout << "STDOUT " << s << std::endl;
    std::cerr << "STDERR " << s << std::endl;
}

Run: `g++ b.cpp -o b.exe`

In [108]:
# интерактивная запускалка для программ
a = TInteractiveLauncher("./b.exe")

In [109]:
a.write("Hello\n")

In [110]:
a.close()